In [ ]:
# DRPC_ENDPOINT = https://lb.drpc.org/ogrpc?network=ethereum&dkey=Ai84fdxnPEUbs_lb0Z5S4xm6v3uvq0IR75a6DonbV6cR
# INFURA_URL = "https://mainnet.infura.io/v3/76cd24ac1bf34056bf2852227c0a5967" 
# Etherscan API Key:7KBD3HZMISDFDTSFMYRWNKE65RM9YVKN7W

In [ ]:
import requests
import json
from web3 import Web3
import pandas as pd
import time

In [ ]:
#Etherscan
ETHERSCAN_API_KEY = '7KBD3HZMISDFDTSFMYRWNKE65RM9YVKN7W'
ETHERSCAN_API_URL = "https://api.etherscan.io/api"

#Alternative: dRPC
DRPC_ENDPOINT = "https://lb.drpc.org/ogrpc?network=ethereum&dkey=Ai84fdxnPEUbs_lb0Z5S4xm6v3uvq0IR75a6DonbV6cR"
web3 = Web3(Web3.HTTPProvider(DRPC_ENDPOINT))

#Alternative: Infura
INFURA_URL = "https://mainnet.infura.io/v3/76cd24ac1bf34056bf2852227c0a5967"

In [ ]:
def check_address_type(address):
    """
    Determine if an address is an EOA or smart contract.
    """
    try:
        checksum_address = web3.to_checksum_address(address)
        code = web3.eth.get_code(checksum_address)
        return "EOA" if code == b'' else "Smart Contract"
    except Exception as e:
        print(f"Error checking address {address}: {e}")
        return "Error"

In [ ]:
# def fetch_all_transfers(source, target):
#     """
#     Fetch all transactions between source and target.
#     Handles both token and Ether transfers.
#     """
#     try:
#         # Convert to checksum addresses
#         source = Web3.to_checksum_address(source)
#         target = Web3.to_checksum_address(target)

#         all_timestamps = []

#         # First, check for token transfers
#         params = {
#             "module": "account",
#             "action": "tokentx",
#             "address": source,
#             "startblock": 0,
#             "endblock": 99999999,
#             "sort": "asc",  # Oldest first
#             "apikey": ETHERSCAN_API_KEY
#         }
#         response = requests.get(ETHERSCAN_API_URL, params=params)
#         if response.status_code == 200:
#             data = response.json()
#             if data["status"] == "1":
#                 for tx in data["result"]:
#                     if tx["contractAddress"].lower() == target.lower():
#                         all_timestamps.append(int(tx["timeStamp"]))
#             else:
#                 print(f"No token transfers found: {data['message']}")

#         # Check for normal Ether transfers
#         params["action"] = "txlist"
#         response = requests.get(ETHERSCAN_API_URL, params=params)
#         if response.status_code == 200:
#             data = response.json()
#             if data["status"] == "1":
#                 for tx in data["result"]:
#                     if tx["to"].lower() == target.lower():
#                         all_timestamps.append(int(tx["timeStamp"]))
#             else:
#                 print(f"No Ether transfers found: {data['message']}")
#         return all_timestamps
#     except Exception as e:
#         print(f"Error fetching transactions for {source} -> {target}: {e}")
#         return []
def fetch_all_transfers(source, target):
    """
    Fetch all transactions between source and target.
    Handles both token and Ether transfers, and checks all cases 
    where the source or target can appear in both 'from' and 'to' fields.
    """
    try:
        # Convert to checksum addresses
        source = Web3.to_checksum_address(source)
        target = Web3.to_checksum_address(target)

        all_timestamps = []

        # First, check for token transfers
        params = {
            "module": "account",
            "action": "tokentx",
            "address": source,
            "startblock": 0,
            "endblock": 99999999,
            "sort": "asc",  # Oldest first
            "apikey": ETHERSCAN_API_KEY
        }
        response = requests.get(ETHERSCAN_API_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            if data["status"] == "1":
                for tx in data["result"]:
                    # Check if the transaction involves the target
                    if (tx["contractAddress"].lower() == target.lower() or
                            tx["to"].lower() == target.lower() or
                            tx["from"].lower() == target.lower()):
                        all_timestamps.append(int(tx["timeStamp"]))
            else:
                print(f"No token transfers found: {data['message']}")

        # Check for normal Ether transfers (including both directions)
        params["action"] = "txlist"
        response = requests.get(ETHERSCAN_API_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            if data["status"] == "1":
                for tx in data["result"]:
                    # Check if the transaction involves the target
                    if (tx["to"].lower() == target.lower() or
                            tx["from"].lower() == target.lower()):
                        all_timestamps.append(int(tx["timeStamp"]))
            else:
                print(f"No Ether transfers found: {data['message']}")

        return all_timestamps
    except Exception as e:
        print(f"Error fetching transactions for {source} -> {target}: {e}")
        return []

In [103]:
# # File paths
input_file = "../dataset/first degree/txd_to_address.csv"
output_file = "../dataset/first degree/txd_with_features.csv"
# input_file = "../dataset/first degree/txd_subset.csv"
# output_file = "../dataset/first degree/txd_with_features.csv"
try:
    df = pd.read_csv(output_file)  # Try to load the output file if it exists
    print(f"Resuming progress from {output_file}")
except FileNotFoundError:
    df = pd.read_csv(input_file)  # Load the original input file if no output file exists
    print(f"Starting fresh with {input_file}")

# Ensure dataset has required columns
required_columns = [
    "source", "target", "latest_timestamp", "average_timestamp",
    "earliest_timestamp", "interaction_duration", "transaction_count"
]
for col in required_columns:
    if col not in df.columns:
        df[col] = None

# Identify rows that still need processing
rows_to_process = df[df["latest_timestamp"].isna()]

# Process each transaction in the dataset
batch_size = 100
processed_count = 0

for idx, row in rows_to_process.iterrows():
    try:
        source = row["source"]
        target = row["target"]
        print(f"Processing transaction {idx+1}/{len(df)}: {source} -> {target}")

        # Fetch all transaction timestamps
        timestamps = fetch_all_transfers(source, target)

        if timestamps:
            df.at[idx, "latest_timestamp"] = max(timestamps)
            df.at[idx, "earliest_timestamp"] = min(timestamps)
            df.at[idx, "average_timestamp"] = sum(timestamps) / len(timestamps) if len(timestamps) > 1 else timestamps[0]
            df.at[idx, "interaction_duration"] = max(timestamps) - min(timestamps)
            df.at[idx, "transaction_count"] = len(timestamps)
        else:
            print(f"No transactions found for {source} -> {target}")

        # Increment processed count
        processed_count += 1

        # Save progress every 100 transactions
        if processed_count % batch_size == 0:
            print(f"Saving progress at {processed_count} transactions...")
            df.to_csv(output_file, index=False)
            print(f"Progress saved to {output_file}")

        # Add a delay to avoid hitting API rate limits
        time.sleep(1)
    except Exception as e:
        print(f"Error processing row {idx+1}: {e}")

# Final save to ensure all rows are written
df.to_csv(output_file, index=False)
print(f"Final dataset saved to {output_file}")

Resuming progress from ../dataset/first degree/txd_with_features.csv
Processing transaction 607/485205: 793ae8c1b1a160bfc07bfb0d04f85eab1a71f4f2 -> 3bbbdca04cbbf6129a50fc35164eeb2a7f9b8550
Error fetching transactions for 0x793AE8C1b1a160BFc07BFB0d04F85eab1A71F4f2 -> 0x3bbbDCa04Cbbf6129A50Fc35164EeB2a7f9B8550: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
No transactions found for 793ae8c1b1a160bfc07bfb0d04f85eab1a71f4f2 -> 3bbbdca04cbbf6129a50fc35164eeb2a7f9b8550
Processing transaction 630/485205: 793ae8c1b1a160bfc07bfb0d04f85eab1a71f4f2 -> 6c8b51ba3ef6445bc26f3641f99a391827b365b6
Processing transaction 721/485205: 793ae8c1b1a160bfc07bfb0d04f85eab1a71f4f2 -> 5ac39bdcdd5ff549636da4bcf7d5b6d348f19d52
Processing transaction 722/485205: 793ae8c1b1a160bfc07bfb0d04f85eab1a71f4f2 -> 994e45646d2f10adec526e4dc0988d90b8cde9cc
Processing transaction 723/485205: 793ae8c1b1a160bfc07bfb0d04f85eab1a71f4f2 -> 27270e8d3247299fa3b9c8a9825a37fdc3329328
Processing transacti

KeyboardInterrupt: 

In [ ]:
def fetch_transfers_batch(sources, targets):
    """
    Fetch transactions for up to 5 source-target pairs in a single batch.
    """
    all_timestamps = [None] * len(sources)  # Initialize timestamps for all pairs
    try:
        for i, (source, target) in enumerate(zip(sources, targets)):
            # Convert to checksum addresses
            source = Web3.to_checksum_address(source)
            target = Web3.to_checksum_address(target)

            timestamps = []

            # Check for token transfers
            params = {
                "module": "account",
                "action": "tokentx",
                "address": source,
                "startblock": 0,
                "endblock": 99999999,
                "sort": "asc",  # Oldest first
                "apikey": ETHERSCAN_API_KEY
            }
            response = requests.get(ETHERSCAN_API_URL, params=params)
            if response.status_code == 200:
                data = response.json()
                if data["status"] == "1":
                    for tx in data["result"]:
                        if (tx["contractAddress"].lower() == target.lower() or
                                tx["to"].lower() == target.lower() or
                                tx["from"].lower() == target.lower()):
                            timestamps.append(int(tx["timeStamp"]))

            # Check for Ether transfers (including both directions)
            params["action"] = "txlist"
            response = requests.get(ETHERSCAN_API_URL, params=params)
            if response.status_code == 200:
                data = response.json()
                if data["status"] == "1":
                    for tx in data["result"]:
                        if (tx["to"].lower() == target.lower() or
                                tx["from"].lower() == target.lower()):
                            timestamps.append(int(tx["timeStamp"]))

            if timestamps:
                all_timestamps[i] = timestamps
    except Exception as e:
        print(f"Error in batch processing: {e}")
    return all_timestamps


# File paths
input_file = "../dataset/first degree/txd_to_address.csv"
output_file = "../dataset/first degree/txd_with_features.csv"

# Load the dataset
try:
    df = pd.read_csv(output_file)  # Try to load the output file if it exists
    print(f"Resuming progress from {output_file}")
except FileNotFoundError:
    df = pd.read_csv(input_file)  # Load the original input file if no output file exists
    print(f"Starting fresh with {input_file}")

# Ensure dataset has required columns
required_columns = [
    "source", "target", "latest_timestamp", "average_timestamp",
    "earliest_timestamp", "interaction_duration", "transaction_count"
]
for col in required_columns:
    if col not in df.columns:
        df[col] = None

# Identify rows that still need processing
rows_to_process = df[df["latest_timestamp"].isna()]

# Process transactions in batches of 5
batch_size = 5
processed_count = 0

for batch_start in range(0, len(rows_to_process), batch_size):
    batch_end = min(batch_start + batch_size, len(rows_to_process))
    batch = rows_to_process.iloc[batch_start:batch_end]

    sources = batch["source"].tolist()
    targets = batch["target"].tolist()
    indices = batch.index.tolist()

    print(f"Processing transactions {', '.join(map(str, indices))} / {len(df)}...")

    # Fetch all transaction timestamps for the batch
    batch_timestamps = fetch_transfers_batch(sources, targets)

    for i, (idx, timestamps) in enumerate(zip(batch.index, batch_timestamps)):
        if timestamps:
            df.at[idx, "latest_timestamp"] = max(timestamps)
            df.at[idx, "earliest_timestamp"] = min(timestamps)
            df.at[idx, "average_timestamp"] = sum(timestamps) / len(timestamps) if len(timestamps) > 1 else timestamps[0]
            df.at[idx, "interaction_duration"] = max(timestamps) - min(timestamps)
            df.at[idx, "transaction_count"] = len(timestamps)
        else:
            print(f"No transactions found for {sources[i]} -> {targets[i]}")

        # Increment processed count
        processed_count += 1

    # Save progress every 100 transactions
    if processed_count % 100 == 0:
        print(f"Saving progress at {processed_count} transactions...")
        df.to_csv(output_file, index=False)
        print(f"Progress saved to {output_file}")

    # Add a delay to avoid hitting API rate limits
    time.sleep(1)

# Final save to ensure all rows are written
df.to_csv(output_file, index=False)
print(f"Final dataset saved to {output_file}")

Resuming progress from ../dataset/first degree/txd_with_features.csv
Processing transactions 2055, 2134, 2137, 2138, 2143 / 485205...
No transactions found for 5fe5b7546d1628f7348b023a0393de1fc825a4fd -> bcd159845b81e35a0b5aff57feccd0b3114791ab
No transactions found for 07307d0b136a79bac718f43388aed706389c4588 -> 90bc9dc09464e651c47dd06b4bf1c5f843251a14
No transactions found for 07307d0b136a79bac718f43388aed706389c4588 -> 1719ebdc0646fbf268b4feeb57af842ffbe4d45f
No transactions found for 6c8f2a135f6ed072de4503bd7c4999a1a17f824b -> a9afbdd1bc69051d955d5afefc896e903460f547
Processing transactions 2144, 2145, 2146, 2147, 2148 / 485205...
No transactions found for 6c8f2a135f6ed072de4503bd7c4999a1a17f824b -> 3c11aec47b707cd67b042f52d8df050d36f3aeae
No transactions found for 6c8f2a135f6ed072de4503bd7c4999a1a17f824b -> c9bc1f028f960a70c7e47021be439c816ad62d6f
No transactions found for 6c8f2a135f6ed072de4503bd7c4999a1a17f824b -> 6040f4e582a948a9b555932455d484a7cd3827d0
No transactions found fo